<a href="https://colab.research.google.com/github/Jyotiranisahoo/Data-Science-Internship-Project/blob/main/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("https://raw.githubusercontent.com/Premalatha-success/Datasets/main/h1n1_vaccine_prediction.csv")

In [ ]:
df.head()

,unique_id,h1n1_worry,h1n1_awareness,antiviral_medication,contact_avoidance,bought_face_mask,wash_hands_frequently,avoid_large_gatherings,reduced_outside_home_cont,avoid_touch_face,...,race,sex,income_level,marital_status,housing_status,employment,census_msa,no_of_adults,no_of_children,h1n1_vaccine
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,White,Female,Below Poverty,Not Married,Own,Not in Labor Force,Non-MSA,0.0,0.0,0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,White,Male,Below Poverty,Not Married,Rent,Employed,"MSA, Not Principle City",0.0,0.0,0
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,"MSA, Not Principle City",2.0,0.0,0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,White,Female,Below Poverty,Not Married,Rent,Not in Labor Force,"MSA, Principle City",0.0,0.0,0
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,"MSA, Not Principle City",1.0,0.0,0


In [ ]:
### problem statement https://www.kaggle.com/competitions/prediction-of-h1n1-vaccination/data

In [ ]:
import numpy as np
import pandas as pd
import os

path=('/kaggle/input/prediction-of-h1n1-vaccination/')
train = pd.read_csv(path +'train.csv')
train_labels = pd.read_csv(path +'train_labels.csv')
test = pd.read_csv(path +'test.csv')
submission = pd.read_csv(path + 'submission.csv')

In [ ]:
#The light gbm model is capable of producing a warning message, but it still work
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
print(f'train size :\t\t{train.shape}')
print(f'train_labels size :\t{train_labels.shape}')
print(f'test size :\t\t{test.shape}')
print(f'submission size :\t{submission.shape}')

In [ ]:
# The "train_labels" contain our target variable: "vacc_h1n1_f"
train_labels

In [ ]:
submission

In [ ]:
# We can observe a target value ratio of 3:1.
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

def write_percent(ax, total_size):
    for patch in ax.patches:
        height = patch.get_height()
        width = patch.get_width()
        left_coord = patch.get_x()
        percent = height / total_size*100
        
        ax.text(x=left_coord + width/2,
                y = height + total_size * 0.001,
                s = f'{percent:1.1f}%',
                ha = 'center')
        
plt.figure(figsize = (7, 6))
ax = sns.countplot(x='vacc_h1n1_f', data = train_labels, palette='Set3')
ax.set_title('Distribution of Target Variable')
write_percent(ax, len(train_labels))


In [ ]:
# merge train data, test data
data = pd.concat([train, test], ignore_index = True)
print(f'train size :\t{train.shape}')
print(f'test size :\t{test.shape}')
print(f'merged size :\t{data.shape}')

In [ ]:
# create a table that displays both the unique data and the missing values in a dataset.
def resumetable(df):
    print(f'dataset shape: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['data type'])
    summary = summary.rename(columns = {'index' : 'feature'})
    summary['number of missing value'] = df.isnull().sum().values
    summary['number of unique data'] = df.nunique().values
    summary['first value'] = df.iloc[0,:]
    summary['second value'] = df.iloc[1,:]
    summary['third value'] = df.iloc[2,:]
    
    return summary

resumetable(data)
dataset shape: (70258, 38)

In [ ]:
# there are a lot of missing values, but I won't handle it this time.
missing = pd.DataFrame(data.isna().mean().multiply(100).round(2), columns=['percent'])
missing

In [ ]:
import matplotlib.gridspec as gridspec

def plot_target_ratio_by_features(df, features, num_rows, num_cols, size=(12,18)):
    mpl.rc('font', size=9)
    plt.figure(figsize = size)
    grid = gridspec.GridSpec(num_rows, num_cols)
    plt.subplots_adjust(wspace = 0.5, hspace = 0.5)
    
    for idx, feature in enumerate(features):
        ax = plt.subplot(grid[idx])
        sns.barplot(x=feature, y=train_labels['vacc_h1n1_f'], data = df, palette = 'Set3', ax= ax)

In [ ]:
# Categorize data types
ordinal_vars = ['h1n1_concern', 'h1n1_knowledge', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
                'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective', 'opinion_seas_risk',
                'opinion_seas_sick_from_vacc', 'agegrp', 'education_comp', 'inc_pov']

binary_vars = ['behavioral_antiviral_meds', 'behavioral_avoidance', 'behavioral_face_mask',
               'behavioral_wash_hands', 'behavioral_large_gatherings', 'behavioral_outside_home',
               'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal', 'chronic_med_condition',
               'child_under_6_months', 'health_insurance', 'health_worker', 'sex_i', 'marital', 'rent_own_r',
               'census_msa']

categorical_vars = ['raceeth4_i', 'employment_status', 'census_region', 'employment_industry',
                    'employment_occupation', 'hhs_region', 'state']

numerical_vars = ['n_adult_r', 'household_children', 'n_people_r']

In [ ]:
plot_target_ratio_by_features(data, ordinal_vars, 4, 3)
# We will delete below feature due to issues with the confidence intervals.
# opinion_h1n1_sick_from_vacc
# opinion_seas_vacc_effective
# opinion_seas_sick_from_vacc

In [ ]:
 plot_target_ratio_by_features(data, binary_vars, 6, 3)
# We will delete below feature because there were issues with the similarity ratio of the target
# sex_i 

# not binary data
# rent_own_r 
# census_msa binary

In [ ]:
plot_target_ratio_by_features(data, categorical_vars, 3, 3)
# We will delete below feature due to issues with the confidence intervals.
# employment_industry 

In [ ]:
plot_target_ratio_by_features(data, numerical_vars, 3, 3)

In [ ]:
# merge data 
data_ordinal = data.loc[:,ordinal_vars].copy()
data_binary = data.loc[:,binary_vars].copy()
data_categorical = data.loc[:,categorical_vars].copy()
data_numerical = data.loc[:,numerical_vars].copy()
data_sorted = pd.concat([data_ordinal, data_binary, data_categorical, data_numerical], axis=1)
resumetable(data_sorted)

In [ ]:
remove_list = ['opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective', 'opinion_seas_sick_from_vacc',
               'rent_own_r', 'census_msa', 'sex_i', 'employment_industry']

data = data_sorted.drop(remove_list, axis = 1)
resumetable(data)

In [ ]:
# ordinal data - ordinal encoding
from sklearn.preprocessing import OrdinalEncoder

ordinal_list = data.columns[0:8].tolist()
ordianl_encoder = OrdinalEncoder()

ordinal_data_encoded = ordianl_encoder.fit_transform(data[ordinal_list])
ordinal_data_encoded

NameError: ignored

In [ ]:
# binary data - minmax encoding
from sklearn.preprocessing import MinMaxScaler

binary_list = data.columns[8:22].tolist()
binary_encoder = MinMaxScaler()

binary_data_encoded = binary_encoder.fit_transform(data[binary_list])
binary_data_encoded

NameError: ignored

In [ ]:
# categorical data - onehot encoding
from sklearn.preprocessing import OneHotEncoder

categoty_list = data.columns[22:28].tolist()
onehot_encoder = OneHotEncoder()

categorical_data_encoded = onehot_encoder.fit_transform(data[categoty_list])
categorical_data_encoded

NameError: ignored

In [ ]:
# numerical data - ordinal encoding
from sklearn.preprocessing import OrdinalEncoder

numerical_list = data.columns[28:].tolist()
ordianl_encoder = OrdinalEncoder()

numerical_data_encoded = ordianl_encoder.fit_transform(data[numerical_list])
numerical_data_encoded

NameError: ignored

In [ ]:
# To improve memory efficiency, it is recommended to create a sparse matrix
from scipy import sparse
all_data_sparse = sparse.hstack([sparse.csr_matrix(ordinal_data_encoded),
                                 sparse.csr_matrix(binary_data_encoded),
                                categorical_data_encoded,
                                sparse.csr_matrix(numerical_data_encoded)],
                               format='csr')
all_data_sparse

NameError: ignored

In [ ]:
# split train, test data
num_train = len(train)

X = all_data_sparse[:num_train]
X_test = all_data_sparse[num_train:]

y = train_labels['vacc_h1n1_f'].values

print(f'X size :{X.shape}')
print(f'X_test size :{X_test.shape}')

NameError: ignored

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                     test_size = 0.1,
                                                     random_state = 97)

bayes_dtrain = lgb.Dataset(X_train, y_train)
bayes_dvalid = lgb.Dataset(X_valid, y_valid)


NameError: ignored

In [ ]:
# basian optimization hyper parameter bounds
param_bounds = {'num_leaves': (38, 42),
                'lambda_l1': (0.68, 0.72),
                'lambda_l2': (0.93, 1),
                'feature_fraction': (0.65, 0.73),
                'bagging_fraction': (0.68, 0.72),
                'min_child_samples': (9, 11),
                'min_child_weight': (25, 35)}

fixed_params = {'objective': 'binary',
                'learning_rate': 0.1,
                'bagging_freq': 1,
                'force_row_wise': True,
                'random_state': 97}

In [ ]:
from sklearn.metrics import f1_score

def f1_score_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    y_pred = np.round(y_pred)
    return 'f1_score', f1_score(y_true, y_pred), True

def eval_function(num_leaves, lambda_l1, lambda_l2, feature_fraction,
                  bagging_fraction, min_child_samples, min_child_weight):
    
    params = {'num_leaves': int(round(num_leaves)),
              'lambda_l1': lambda_l1,
              'lambda_l2': lambda_l2,
              'feature_fraction': feature_fraction,
              'bagging_fraction': bagging_fraction,
              'min_child_samples': int(round(min_child_samples)),
              'min_child_weight': min_child_weight,
              'feature_pre_filter': False}
    # add fixed hyperparameters
    params. update(fixed_params)
    
    print('Hyperparameters:', params)
    
    # Train the LightGBM model
    lgb_model = lgb.train(params=params,
                           train_set=bayes_dtrain,
                           num_boost_round=2500,
                           valid_sets=bayes_dvalid,
                           feval=f1_score_eval,
                           early_stopping_rounds=500,
                           verbose_eval=100)

    # Make predictions with validation data
    preds = lgb_model.predict(X_valid)
    # Calculate the F1-score
    f1_score_val = f1_score(y_valid, np.round(preds))
    print(f'F1-score: {f1_score_val}\n')
    
    return f1_score_val

In [ ]:
from bayes_opt import BayesianOptimization

optimizer = BayesianOptimization(f=eval_function,      
                                 pbounds=param_bounds,
                                 random_state=97)

optimizer.maximize(init_points=5, n_iter=10)

In [ ]:
max_params = optimizer.max['params']
max_params

In [ ]:
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))

In [ ]:
max_params

In [ ]:
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=97)

# This one-dimensional array will be used as valid target values later.
oof_val_preds = np.zeros(X.shape[0]) 
# This one-dimensional array will be used as test target values later.
oof_test_preds = np.zeros(X_test.shape[0]) 

for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    print('#'*33, f'Fold {idx+1} / Fold {folds.n_splits}', '#'*33)
    
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]

    # Create a Dataset for LightGBM
    dtrain = lgb.Dataset(X_train, y_train)
    dvalid = lgb.Dataset(X_valid, y_valid)
                          
    lgb_model = lgb.train(params=max_params,   
                          train_set=dtrain,     
                          num_boost_round=2500, 
                          valid_sets=dvalid,    
                          feval=f1_score_eval,
                          early_stopping_rounds=500, 
                          verbose_eval=100)
    
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)
    
    f1_score_val = f1_score(y_valid, np.round(oof_val_preds[valid_idx]))
    print(f'F1-score: {f1_score_val}\n')

In [ ]:
print('OOF data lgb F1-score :\t', f1_score(y, np.round(oof_val_preds)))

In [ ]:
y_preds_binary = [1 if pred >= 0.5 else 0 for pred in oof_test_preds]

submission['vacc_h1n1_f'] = y_preds_binary
submission.to_csv('submission.csv', index = False)

In [ ]:
submission